# Semi-supervised node classification using Graph Neural Networks

In this tutorial, you will

* Introduce a popular citation network in DGL
* Build a GCN model, a popular Graph Neural Network architecture proposed by [Kipf et al.](https://arxiv.org/abs/1609.02907)
* Train the model and understand the result.

In [1]:
import dgl
import torch
import torch.nn as nn 
import torch.nn.functional as F
import itertools
import numpy as np
import time

Using backend: pytorch


## Semi-supervised node classification: problem statement

- Given the graph structure, node features of all nodes, and node labels on a subset of nodes
- Predict the labels on the rest of the nodes

## Classification in citation networks

- Load the graph data the Cora dataset
- Consists of 2708 scientific publications classified into one of seven classes. 
- The citation network consists of 10556 links. 
- Each paper has a word vector that encodes its abstract as a feature.

<img src='https://miro.medium.com/max/1400/1*oygeCjtUsS87duvFoDT8tA.png' align='center' width="400px" height="300px" />


In [2]:
from dgl.data import CoraGraphDataset
# ----------- 0. load graph -------------- #
data = CoraGraphDataset()
g = data[0]
print(g)

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={})


In [3]:
# Print data attributes. Note that these are attributes of the graph
print(g.ndata.keys())
print(g.ndata)

dict_keys(['train_mask', 'val_mask', 'test_mask', 'label', 'feat'])
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}


In [4]:
    
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    in_feats = features.shape[1]
    n_classes = data.num_labels
    n_edges = data.graph.number_of_edges()
    print("""----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (n_edges, n_classes,
              train_mask.int().sum().item(),
              val_mask.int().sum().item(),
              test_mask.int().sum().item()))


----Data statistics------'
  #Edges 10556
  #Classes 7
  #Train samples 140
  #Val samples 500
  #Test samples 1000


/home/ubuntu/.local/lib/python3.6/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/ubuntu/.local/lib/python3.6/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.graph will be deprecated, please use dataset[0] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


## Define a GCN model

- Each layer computes new node representations by aggregating neighbor information as follows

$$
 h_i^{(l+1)} = \sigma(b^{(l)} + \sum_{j\in\mathcal{N}(i)}\frac{1}{c_{ij}}h_j^{(l)}W^{(l)})
$$
<img src='https://tkipf.github.io/graph-convolutional-networks/images/gcn_web.png' align='center' width="400px" height="300px" />

### Message passing interpretation of GCN

- Aggregate the messages from neighbors and update
![Imgur](msp.png)

### GCN  update functions
![Imgur](gcnmsp.png)


### GCN layer in DGL 

- The GCN layer uses message passing to implement the previous functions as follows
- The reduce function sums 
- The projection matrix is applied in the end

In [5]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            gcn_msg = fn.copy_src(src='h', out='m')
            gcn_reduce = fn.sum(msg='m', out='h')
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)


### GraphConv implementation of GCN

- DGL provides implementation of many popular neighbor aggregation modules.
- GCN layer is implemented using the GraphConv layer
- See the full list of supported [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv).

In [6]:
from dgl.nn import GraphConv

# ----------- 2. create model -------------- #
# build a two-layer GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# Create the model with given dimensions 
# input layer dimension: 1433, node features
# hidden layer dimension: 16
# output layer dimension: n_classes
model = GCN(in_feats, 16, n_classes)

In [7]:
def evaluate(g,model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g,features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


In [8]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters()), lr=0.01)
loss_fcn = torch.nn.CrossEntropyLoss()
# ----------- 4. training -------------------------------- #
n_epochs=200
for epoch in range(n_epochs):
        model.train()

        # forward
        logits = model(g,features)
        loss = loss_fcn(logits[train_mask], labels[train_mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        acc = evaluate(g,model, features, labels, val_mask)
        if epoch%20==0:
            print("Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} | ". format(epoch, loss.item(),
                                             acc))
print()


Epoch 00000 | Loss 1.9459 | Accuracy 0.2700 | 
Epoch 00020 | Loss 1.5864 | Accuracy 0.6940 | 
Epoch 00040 | Loss 0.9247 | Accuracy 0.7320 | 
Epoch 00060 | Loss 0.4107 | Accuracy 0.7660 | 
Epoch 00080 | Loss 0.1838 | Accuracy 0.7700 | 
Epoch 00100 | Loss 0.0946 | Accuracy 0.7720 | 
Epoch 00120 | Loss 0.0565 | Accuracy 0.7740 | 
Epoch 00140 | Loss 0.0378 | Accuracy 0.7680 | 
Epoch 00160 | Loss 0.0273 | Accuracy 0.7640 | 
Epoch 00180 | Loss 0.0209 | Accuracy 0.7660 | 



In [9]:
# ----------- 5. check results ------------------------ #
acc = evaluate(g,model, features, labels, test_mask)
print("Test accuracy {:.2%}".format(acc))

Test accuracy 76.00%
